In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Dataset/creditcard.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score, precision_recall_curve


In [4]:
X = df.drop('Class', axis=1)
y = df['Class']

scaler = StandardScaler()
X[['Time', 'Amount']] = scaler.fit_transform(X[['Time', 'Amount']])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
model_bal = LogisticRegression(max_iter=1000, class_weight='balanced')
model_bal.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [6]:
y_pred_bal = model_bal.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_bal))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_bal))

y_scores_bal = model_bal.predict_proba(X_test)[:,1]
ap_bal = average_precision_score(y_test, y_scores_bal)
print("Average Precision (PR-AUC):", ap_bal)


Confusion Matrix:
[[55475  1389]
 [    8    90]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962

Average Precision (PR-AUC): 0.7189463752156904


After introducing cost-sensitive learning using class weights, the model’s behavior changed significantly. Recall for fraud increased from about 64% to over 92%, meaning we now catch almost all fraud cases. However, this comes at the cost of very low precision, so many legitimate transactions are flagged. This demonstrates the fundamental trade-off in fraud systems: we can reduce fraud losses or reduce customer friction, but not both simultaneously. The model itself is good at ranking transactions, as shown by a PR-AUC of about 0.72, so the next step is to tune the decision threshold to find a business-optimal operating point